<a href="https://colab.research.google.com/github/navneetslovely/QML_genom/blob/main/QSVC_Pauli.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Quantum_Studies_phd/PhD/QML_Genom
!pip install qiskit-machine-learning
!pip install qiskit[visualization]
!pip install genomic-benchmarks
!pip install tensorflow>=2.6.0
!pip install tensorflow-addons
!pip install typing-extensions --upgrade
!pip install qiskit-aer

MessageError: Error: credential propagation was unsuccessful

In [ ]:
from pathlib import Path
import os
import sys
import tensorflow as tf
import warnings
import numpy as np
warnings.filterwarnings("ignore")
if not sys.warnoptions:
    warnings.simplefilter("ignore")
from genomic_benchmarks.loc2seq import download_dataset
from genomic_benchmarks.data_check import is_downloaded, info
from genomic_benchmarks.models.tf import vectorize_layer
from genomic_benchmarks.models.tf import get_basic_cnn_model_v0 as get_model
import matplotlib.pyplot as plt
import time
from qiskit.circuit.library import ZZFeatureMap, RealAmplitudes,ZFeatureMap, PauliFeatureMap
from qiskit_algorithms.optimizers import COBYLA
from qiskit_machine_learning.algorithms.classifiers import VQC
from functools import partial
# from qiskit import Aer
from qiskit_machine_learning.circuit.library import RawFeatureVector
from qiskit.primitives import Sampler
import pickle
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit_machine_learning.algorithms import PegasosQSVC
import json
import time
import warnings
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import clear_output
from qiskit import ClassicalRegister, QuantumRegister
from qiskit import QuantumCircuit
from qiskit.circuit.library import RealAmplitudes
from qiskit.quantum_info import Statevector
from qiskit_algorithms.optimizers import COBYLA
from qiskit_algorithms.utils import algorithm_globals

from qiskit_machine_learning.circuit.library import RawFeatureVector
from qiskit_machine_learning.neural_networks import SamplerQNN
from sklearn.decomposition import PCA
algorithm_globals.random_seed = 42

In [ ]:
# Load the preprocessed data
train_sequences = np.load('train_sequences.npy')
# print('train_sequences',train_sequences[:2])
train_labels = np.load('train_labels.npy')
# print('train_labels',train_labels[:2])
test_sequences = np.load('test_sequences.npy')
# print('test_sequences',test_sequences[:2])
test_labels = np.load('test_labels.npy')
# print('test_labels',test_labels[:2])
# Perform PCA transformation on the data
pca = PCA(n_components=4)
train_sequences = pca.fit_transform(train_sequences)
test_sequences= pca.fit_transform(test_sequences)

# Split the train_sequences, train_labels, test_sequences, and test_labels into 15 parts each
split_train_sequences = np.array_split(train_sequences, 15)
split_train_labels = np.array_split(train_labels, 15)
split_test_sequences = np.array_split(test_sequences, 15)
split_test_labels = np.array_split(test_labels, 15)

# Create directories to save the parts
os.makedirs('/content/drive/MyDrive/Quantum_Studies_phd/PhD/QML_Genom/QML_genom_seq/train_sequences_parts', exist_ok=True)
os.makedirs('/content/drive/MyDrive/Quantum_Studies_phd/PhD/QML_Genom/QML_genom_seq/train_labels_parts', exist_ok=True)
os.makedirs('/content/drive/MyDrive/Quantum_Studies_phd/PhD/QML_Genom/QML_genom_seq/test_sequences_parts', exist_ok=True)
os.makedirs('/content/drive/MyDrive/Quantum_Studies_phd/PhD/QML_Genom/QML_genom_seq/test_labels_parts', exist_ok=True)

# Save each part into a separate file
for i in range(15):
    np.save(f'/content/drive/MyDrive/Quantum_Studies_phd/PhD/QML_Genom/QML_genom_seq/train_sequences_parts/train_seq_{i+1}.npy', split_train_sequences[i])
    np.save(f'/content/drive/MyDrive/Quantum_Studies_phd/PhD/QML_Genom/QML_genom_seq/train_labels_parts/train_labels_{i+1}.npy', split_train_labels[i])
    np.save(f'/content/drive/MyDrive/Quantum_Studies_phd/PhD/QML_Genom/QML_genom_seq/test_sequences_parts/test_seq_{i+1}.npy', split_test_sequences[i])
    np.save(f'/content/drive/MyDrive/Quantum_Studies_phd/PhD/QML_Genom/QML_genom_seq/test_labels_parts/test_labels_{i+1}.npy', split_test_labels[i])

In [ ]:
# Define the feature dimension or number of qubits
feature_dim = len(train_sequences[0])
print("Feature dimension and number of qubits:", feature_dim)

In [ ]:
from qiskit_algorithms.state_fidelities import ComputeUncompute
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit_machine_learning.algorithms import QSVC

feature_dim = len(train_sequences[0])
feature_map = PauliFeatureMap(feature_dim, reps=2)
sampler = Sampler()
fidelity = ComputeUncompute(sampler=sampler)
qkernel = FidelityQuantumKernel(fidelity=fidelity, feature_map=feature_map)
qsvc = QSVC(kernel=qkernel)


In [ ]:
num_epochs = 1  # Specify the number of epochs
batch_size = 1024  # Specify the batch size
total_time = 0

# Placeholder for saving scores
train_scores = []
test_scores = []
all_part_scores = []

In [ ]:
qsvc.load("/content/drive/MyDrive/Quantum_Studies_phd/PhD/QML_Genom/QSVC_PauliFeatureMap_after_part_1_epoch_1")

In [ ]:
# num_epochs = 10  # Specify the number of epochs
for epoch in range(num_epochs):
    start_time = time.time()  # Start time for the epoch
    epoch_train_scores = []  # Store training scores for each epoch

    # Loop over each data split
    for a in range(2, 5):
        # Load the split data
        train_sequences_part = np.load(f'/content/drive/MyDrive/Quantum_Studies_phd/PhD/QML_Genom/QML_genom_seq/train_sequences_parts/train_seq_{a}.npy')
        train_labels_part = np.load(f'/content/drive/MyDrive/Quantum_Studies_phd/PhD/QML_Genom/QML_genom_seq/train_labels_parts/train_labels_{a}.npy')

        print(f"Training on part {a} started")

        # Perform training for each batch
        for i in range(0, len(train_sequences_part), batch_size):
            batch_sequences = train_sequences_part[i:i+batch_size]
            batch_labels = train_labels_part[i:i+batch_size]

            # Fit the model on the batch
            qsvc.fit(batch_sequences, batch_labels)

            # # Optionally evaluate on training data
            # score = qsvc.score(batch_sequences, batch_labels)
            # epoch_train_scores.append(score)

            print(f"Epoch {epoch + 1}, Part {a} - Batch {i // batch_size + 1} completed")

        # Optionally, save the model after each part is processed within an epoch
        qsvc.save(f'QSVC_PauliFeatureMap_after_part_{a}_epoch_{epoch+1}')
        print(f"Model saved after part {a}, epoch {epoch+1} completed")
        with open(f'QSVC_PauliFeatureMap_after_part_{a}_epoch_{epoch+1}.pkl', 'wb') as f:
            pickle.dump(qsvc, f)
        print(f"Pickle Model saved after part {a}, epoch {epoch+1} completed")

        # # File path for training scores
        # part_scores_file = f'QSVC_ZFeatureMap_train_scores_part_{a}_epoch_{epoch+1}.npy'

        # # Check if the file exists and append if it does
        # if os.path.exists(part_scores_file):
        #     existing_scores = np.load(part_scores_file)
        #     updated_scores = np.concatenate((existing_scores, np.array(epoch_train_scores)))
        #     np.save(part_scores_file, updated_scores)
        #     print(f"Scores appended to file {part_scores_file}")
        # else:

        #     np.save(part_scores_file, np.array(epoch_train_scores))

    # Average training score for the epoch
    # epoch_train_scores = np.load(f'/content/drive/MyDrive/Quantum_Studies_phd/PhD/QML_Genom/QML_genom_seq/train_sequences_parts/train_seq_{a}.npy')

    # for a in range(1, 5):
    #     part_scores_file = f'QSVC_ZFeatureMap_train_scores_part_{a}_epoch_{epoch+1}.npy'
    #     if os.path.exists(part_scores_file):
    #         # Load the scores from file
    #         part_scores = np.load(f'/content/drive/MyDrive/Quantum_Studies_phd/PhD/QML_Genom/{part_scores_file}')
    #         # Calculate the average score for this part
    #         average_part_score = np.mean(part_scores)
    #         all_part_scores.append(average_part_score)
    #         print(f"Average score for part {a} in epoch {epoch+1}: {average_part_score:.4f}")

    # # Average training score for the epoch
    # average_train_score = np.mean(all_part_scores)
    # train_scores.append(average_train_score)

    # # Evaluate on the entire test dataset after each epoch
    # test_score = qsvc.score(test_sequences[:4000], test_labels[:4000])
    # test_scores.append(test_score)

    # print(f"Epoch {epoch+1} - Training Score: {average_train_score:.4f}, Test Score: {test_score:.4f}")

    qsvc.save(f'QSVC_PauliFeatureMap_epoch_{epoch+1}')
    print(f"Model saved after epoch {epoch+1} completed")
    # Save the model using pickle
    with open(f'QSVC_PauliFeatureMap_epoch_{epoch+1}.pkl', 'wb') as f:
        pickle.dump(qsvc, f)
    print(f"Model saved after pickle epoch {epoch+1} completed")

    # Calculate the time for the epoch
    end_time = time.time()
    epoch_time = end_time - start_time
    total_time += epoch_time

    print(f"Epoch {epoch+1} - Time taken: {epoch_time:.2f} seconds")

print(f"Total training time: {total_time:.2f} seconds")

# # Save scores to a file for later use
# # np.save('Pegaos_QSVM_train_scores_ZFeatureMap.npy', np.array(train_scores))
# # np.save('Pegaos_QSVM_test_scores_ZFeatureMap.npy', np.array(test_scores))

# train_scores_file = 'QSVC_train_scores_ZFeatureMap.npy'
# test_scores_file = 'QSVC_train_scores_ZFeatureMap.npy'

# if os.path.exists(train_scores_file):
#     existing_train_scores = np.load(train_scores_file)
#     updated_train_scores = np.concatenate((existing_train_scores, train_scores))
#     np.save(train_scores_file, updated_train_scores)
# else:
#     np.save(train_scores_file, np.array(train_scores))

# if os.path.exists(test_scores_file):
#     existing_test_scores = np.load(test_scores_file)
#     updated_test_scores = np.concatenate((existing_test_scores, test_scores))
#     np.save(test_scores_file, updated_test_scores)
# else:
#     np.save(test_scores_file, np.array(test_scores))

In [ ]:
qsvc.load("/content/drive/MyDrive/Quantum_Studies_phd/PhD/QML_Genom/QSVC_PauliFeatureMap_epoch_1")

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import classification_report
predictions = qsvc.predict(test_sequences[:4000])
true_labels = test_labels[:4000]

# Calculate Accuracy
accuracy = accuracy_score(true_labels, predictions)
print(f"Accuracy: {accuracy:.4f}")

# Calculate Precision and Recall
precision = precision_score(true_labels, predictions)
recall = recall_score(true_labels, predictions)
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")

# Calculate F1-Score
f1 = f1_score(true_labels, predictions)
print(f"F1 Score: {f1:.4f}")

# Calculate ROC-AUC Score
# Since it's a binary classification, directly pass the true labels and predictions
roc_auc = roc_auc_score(true_labels, predictions)
print(f"ROC-AUC Score: {roc_auc:.4f}")

# Generate a full classification report
report = classification_report(true_labels, predictions)
print("Classification Report:\n", report)